## USA Flight Delay Analytics (2024): Data Cleaning & Preparation for Dashboarding

**Objective:** Clean and transform raw U.S. flight delay data into a dashboard-ready dataset that powers an interactive **Streamlit** + **Plotly** application.

This notebook performs the following:
1. Load the raw dataset (using a smaller sample for development).
2. Perform an initial exploration to understand structure, data types, and missing values.
3. Standardize and rename columns to business-friendly names.
4. Clean the data (handle missing values, fix data types, and remove unnecessary columns).
5. Engineer features for dashboard interactivity (delay flags, buckets, time dimensions, primary delay cause).
6. Export cleaned datasets to `data/processed/` for dashboard use.

---
### Setup + Configuration

First, we import the necessary Python libraries. We'll use **pandas** for loading the CSV file, cleaning the dataset, and performing feature engineering and **numpy** will be used for numerical operations and efficient conditional logic (e.g., creating boolean delay flags, delay buckets, and a primary delay cause field).

In [78]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)
pd.set_option("display.float_format", "{:.2f}".format)

---
### 1. Data Loading

This project includes two raw datasets:
- `data/raw/flight_data_2024.csv` (full dataset ~1.2GB)
- `data/raw/flight_data_2024_sample.csv` (small sample for development)

During development, we load the sample to validate the cleaning pipeline quickly.  
Once the logic is correct, we will process the full dataset using chunked processing (to avoid memory issues).

Raw data is never modified.  
All outputs are written to `data/processed/`.


In [79]:
RAW_SAMPLE = Path("../data/raw/flight_data_2024_sample.csv")
PROCESSED_DIR = Path("../data/processed")

if not RAW_SAMPLE.exists():
    raise FileNotFoundError(f"Sample dataset not found at: {RAW_SAMPLE}")

df = pd.read_csv(RAW_SAMPLE)
print(f"Loaded: {RAW_SAMPLE} | Rows: {len(df):,} | Cols: {df.shape[1]}")

df.head()

Loaded: ../data/raw/flight_data_2024_sample.csv | Rows: 10,000 | Cols: 35


,year,month,day_of_month,day_of_week,fl_date,op_unique_carrier,op_carrier_fl_num,origin,origin_city_name,origin_state_nm,dest,dest_city_name,dest_state_nm,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2024,4,18,4,2024-04-18,MQ,3535.00,DFW,"Dallas/Fort Worth, TX",Texas,RAP,"Rapid City, SD",South Dakota,1018,1015.00,-3.00,21.00,1036.00,1135.00,4.00,1149,1139.00,-10.00,0,NaN,0,151.00,144.00,119.00,835.00,0,0,0,0,0
1,2024,1,1,1,2024-01-01,AA,148.00,CLT,"Charlotte, NC",North Carolina,PHX,"Phoenix, AZ",Arizona,1637,1633.00,-4.00,14.00,1647.00,1900.00,6.00,1923,1906.00,-17.00,0,NaN,0,286.00,273.00,253.00,1773.00,0,0,0,0,0
2,2024,12,12,4,2024-12-12,9E,5440.00,CHA,"Chattanooga, TN",Tennessee,ATL,"Atlanta, GA",Georgia,1000,952.00,-8.00,13.00,1005.00,1034.00,8.00,1059,1042.00,-17.00,0,NaN,0,59.00,50.00,29.00,106.00,0,0,0,0,0
3,2024,4,8,1,2024-04-08,WN,1971.00,OMA,"Omaha, NE",Nebraska,LAS,"Las Vegas, NV",Nevada,1330,1334.00,4.00,8.00,1342.00,1425.00,6.00,1430,1431.00,1.00,0,NaN,0,180.00,177.00,163.00,1099.00,0,0,0,0,0
4,2024,2,16,5,2024-02-16,WN,862.00,BWI,"Baltimore, MD",Maryland,MYR,"Myrtle Beach, SC",South Carolina,1340,1333.00,-7.00,16.00,1349.00,1505.00,4.00,1510,1509.00,-1.00,0,NaN,0,90.00,96.00,76.00,399.00,0,0,0,0,0


---
### 2. Initial Exploration & Profiling

Before cleaning, we inspect:
- Column names and data types
- Missing values
- Basic distributions of delay fields

This helps identify issues such as:
- Null values in delay columns
- HHMM time formatting
- Redundant or overly operational fields not needed for the dashboard

In [80]:
df.info()
df.isna().sum().sort_values(ascending=False).head(25)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 10000 non-null  int64  
 1   month                10000 non-null  int64  
 2   day_of_month         10000 non-null  int64  
 3   day_of_week          10000 non-null  int64  
 4   fl_date              10000 non-null  object 
 5   op_unique_carrier    10000 non-null  object 
 6   op_carrier_fl_num    10000 non-null  float64
 7   origin               10000 non-null  object 
 8   origin_city_name     10000 non-null  object 
 9   origin_state_nm      10000 non-null  object 
 10  dest                 10000 non-null  object 
 11  dest_city_name       10000 non-null  object 
 12  dest_state_nm        10000 non-null  object 
 13  crs_dep_time         10000 non-null  int64  
 14  dep_time             9884 non-null   float64
 15  dep_delay            9884 non-null   

cancellation_code      9878
actual_elapsed_time     164
arr_delay               164
air_time                164
taxi_in                 127
wheels_on               127
arr_time                127
taxi_out                120
wheels_off              120
dep_time                116
dep_delay               116
crs_elapsed_time          0
crs_arr_time              0
distance                  0
cancelled                 0
carrier_delay             0
weather_delay             0
nas_delay                 0
security_delay            0
diverted                  0
year                      0
month                     0
crs_dep_time              0
dest_state_nm             0
dest_city_name            0
dtype: int64

In [81]:
df.describe(include="all").T.head(30)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
year,10000.00,NaN,NaN,NaN,2024.00,0.00,2024.00,2024.00,2024.00,2024.00,2024.00
month,10000.00,NaN,NaN,NaN,6.61,3.38,1.00,4.00,7.00,10.00,12.00
day_of_month,10000.00,NaN,NaN,NaN,15.84,8.79,1.00,8.00,16.00,23.00,31.00
day_of_week,10000.00,NaN,NaN,NaN,3.95,2.01,1.00,2.00,4.00,6.00,7.00
fl_date,10000,366,2024-08-09,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN
op_unique_carrier,10000,15,WN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
op_carrier_fl_num,10000.00,NaN,NaN,NaN,2529.76,1656.00,1.00,1172.00,2254.00,3742.00,8771.00
origin,10000,284,ATL,496,NaN,NaN,NaN,NaN,NaN,NaN,NaN
origin_city_name,10000,278,"Chicago, IL",516,NaN,NaN,NaN,NaN,NaN,NaN,NaN
origin_state_nm,10000,52,California,1068,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---
### 3. Rename Columns

The raw dataset uses DOT/BTS-style column names (technical and cryptic).  
For the dashboard, we rename columns to clear, descriptive names and include units where relevant (e.g., `_min` for minutes).

Examples:
- `op_unique_carrier` → `operating_airline`
- `arr_delay` → `arrival_delay_raw_min`
- `crs_dep_time` → `scheduled_departure_hhmm`

In [82]:
rename_map = {
    "fl_date": "flight_date",
    "op_unique_carrier": "operating_airline",
    "op_carrier_fl_num": "operating_flight_number",

    "origin": "origin_airport",
    "origin_city_name": "origin_city",
    "origin_state_nm": "origin_state",

    "dest": "destination_airport",
    "dest_city_name": "destination_city",
    "dest_state_nm": "destination_state",

    "crs_dep_time": "scheduled_departure_hhmm",
    "crs_arr_time": "scheduled_arrival_hhmm",

    "dep_delay": "departure_delay_raw_min",
    "arr_delay": "arrival_delay_raw_min",

    "carrier_delay": "carrier_delay_min",
    "weather_delay": "weather_delay_min",
    "nas_delay": "nas_delay_min",
    "security_delay": "security_delay_min",
    "late_aircraft_delay": "late_aircraft_delay_min",

    "cancelled": "is_cancelled",
    "diverted": "is_diverted",
}

df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})
df.head()

,year,month,day_of_month,day_of_week,flight_date,operating_airline,operating_flight_number,origin_airport,origin_city,origin_state,destination_airport,destination_city,destination_state,scheduled_departure_hhmm,dep_time,departure_delay_raw_min,taxi_out,wheels_off,wheels_on,taxi_in,scheduled_arrival_hhmm,arr_time,arrival_delay_raw_min,is_cancelled,cancellation_code,is_diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance,carrier_delay_min,weather_delay_min,nas_delay_min,security_delay_min,late_aircraft_delay_min
0,2024,4,18,4,2024-04-18,MQ,3535.00,DFW,"Dallas/Fort Worth, TX",Texas,RAP,"Rapid City, SD",South Dakota,1018,1015.00,-3.00,21.00,1036.00,1135.00,4.00,1149,1139.00,-10.00,0,NaN,0,151.00,144.00,119.00,835.00,0,0,0,0,0
1,2024,1,1,1,2024-01-01,AA,148.00,CLT,"Charlotte, NC",North Carolina,PHX,"Phoenix, AZ",Arizona,1637,1633.00,-4.00,14.00,1647.00,1900.00,6.00,1923,1906.00,-17.00,0,NaN,0,286.00,273.00,253.00,1773.00,0,0,0,0,0
2,2024,12,12,4,2024-12-12,9E,5440.00,CHA,"Chattanooga, TN",Tennessee,ATL,"Atlanta, GA",Georgia,1000,952.00,-8.00,13.00,1005.00,1034.00,8.00,1059,1042.00,-17.00,0,NaN,0,59.00,50.00,29.00,106.00,0,0,0,0,0
3,2024,4,8,1,2024-04-08,WN,1971.00,OMA,"Omaha, NE",Nebraska,LAS,"Las Vegas, NV",Nevada,1330,1334.00,4.00,8.00,1342.00,1425.00,6.00,1430,1431.00,1.00,0,NaN,0,180.00,177.00,163.00,1099.00,0,0,0,0,0
4,2024,2,16,5,2024-02-16,WN,862.00,BWI,"Baltimore, MD",Maryland,MYR,"Myrtle Beach, SC",South Carolina,1340,1333.00,-7.00,16.00,1349.00,1505.00,4.00,1510,1509.00,-1.00,0,NaN,0,90.00,96.00,76.00,399.00,0,0,0,0,0


---
### 4. Data Cleaning

In this section we prepare the dataset for analysis and dashboard performance.

#### 4.1 Drop Unnecessary Columns

To keep the dashboard fast and the field list clean, we remove columns that are:
- Too operational (e.g., wheels on/off, taxi times)
- Too granular (e.g., specific flight number)
- Redundant (e.g., precomputed year/month fields that we can derive from `flight_date`)

This reduces file size and improves dashboard performance.

In [83]:
drop_cols = [
    "operating_flight_number", "dep_time", "arr_time", "wheels_off", "wheels_on", "taxi_out", "taxi_in", "air_time", "distance", 
    "crs_elapsed_time", "actual_elapsed_time", "cancellation_code", "year", "month", "day_of_month", "day_of_week"
]

before_cols = df.shape[1]
df = df.drop(columns=[c for c in drop_cols if c in df.columns])
after_cols = df.shape[1]
print(f"Dropped {before_cols - after_cols} columns. Remaining columns: {after_cols}")

Dropped 16 columns. Remaining columns: 19


#### 4.2 Fix Data Types & Handle Missing Values
We convert dates and ensure numeric delay fields are consistent and safe to aggregate.

In [84]:
# Parse date
df["flight_date"] = pd.to_datetime(df["flight_date"], errors="coerce")

# Convert delay columns to numeric (safe for aggregation)
delay_cols = [
    "departure_delay_raw_min",
    "arrival_delay_raw_min",
    "carrier_delay_min",
    "weather_delay_min",
    "nas_delay_min",
    "security_delay_min",
    "late_aircraft_delay_min"
]

for c in delay_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

# Drop rows missing core dimensions needed for filtering/mapping
core_dims = ["operating_airline", "origin_airport", "origin_state", "flight_date"]
df = df.dropna(subset=[c for c in core_dims if c in df.columns])

df.head()

,flight_date,operating_airline,origin_airport,origin_city,origin_state,destination_airport,destination_city,destination_state,scheduled_departure_hhmm,departure_delay_raw_min,scheduled_arrival_hhmm,arrival_delay_raw_min,is_cancelled,is_diverted,carrier_delay_min,weather_delay_min,nas_delay_min,security_delay_min,late_aircraft_delay_min
0,2024-04-18,MQ,DFW,"Dallas/Fort Worth, TX",Texas,RAP,"Rapid City, SD",South Dakota,1018,-3.00,1149,-10.00,0,0,0,0,0,0,0
1,2024-01-01,AA,CLT,"Charlotte, NC",North Carolina,PHX,"Phoenix, AZ",Arizona,1637,-4.00,1923,-17.00,0,0,0,0,0,0,0
2,2024-12-12,9E,CHA,"Chattanooga, TN",Tennessee,ATL,"Atlanta, GA",Georgia,1000,-8.00,1059,-17.00,0,0,0,0,0,0,0
3,2024-04-08,WN,OMA,"Omaha, NE",Nebraska,LAS,"Las Vegas, NV",Nevada,1330,4.00,1430,1.00,0,0,0,0,0,0,0
4,2024-02-16,WN,BWI,"Baltimore, MD",Maryland,MYR,"Myrtle Beach, SC",South Carolina,1340,-7.00,1510,-1.00,0,0,0,0,0,0,0


---
### 5. Feature Engineering

We create features that power an interactive **Streamlit** + **Plotly** dashboard.

#### 5.1 Calendar, Time-of-Day, and Mapping Features
- Derive year/month/day fields from `flight_date`
- Parse HHMM scheduled departure time into an hour field
- Create state abbreviations (`origin_state_abbr`) for Plotly USA maps

In [85]:
# Calendar fields
df["year"] = df["flight_date"].dt.year.astype("Int64")
df["month"] = df["flight_date"].dt.month.astype("Int64")
df["month_name"] = df["flight_date"].dt.month_name()
df["day_of_month"] = df["flight_date"].dt.day.astype("Int64")
df["day_of_week_name"] = df["flight_date"].dt.day_name()
df["week_of_year"] = df["flight_date"].dt.isocalendar().week.astype("Int64")

# HHMM -> hour + HH:MM
def hhmm_to_hour_min(series: pd.Series):
    s = series.apply(lambda x: np.nan if pd.isna(x) else str(int(float(x))).zfill(4))
    hour = s.apply(lambda x: np.nan if pd.isna(x) else int(x[:2])).astype("Int64")
    minute = s.apply(lambda x: np.nan if pd.isna(x) else int(x[2:])).astype("Int64")
    return hour, minute

if "scheduled_departure_hhmm" in df.columns:
    h, m = hhmm_to_hour_min(df["scheduled_departure_hhmm"])
    df["scheduled_departure_hour"] = h
    df["scheduled_departure_time"] = [
        np.nan if pd.isna(hh) or pd.isna(mm) else f"{int(hh):02d}:{int(mm):02d}"
        for hh, mm in zip(h.tolist(), m.tolist())
    ]

# State abbreviations for Plotly USA maps
US_STATE_TO_ABBR = {
    "Alabama":"AL","Alaska":"AK","Arizona":"AZ","Arkansas":"AR","California":"CA","Colorado":"CO",
    "Connecticut":"CT","Delaware":"DE","District of Columbia":"DC","Florida":"FL","Georgia":"GA",
    "Hawaii":"HI","Idaho":"ID","Illinois":"IL","Indiana":"IN","Iowa":"IA","Kansas":"KS","Kentucky":"KY",
    "Louisiana":"LA","Maine":"ME","Maryland":"MD","Massachusetts":"MA","Michigan":"MI","Minnesota":"MN",
    "Mississippi":"MS","Missouri":"MO","Montana":"MT","Nebraska":"NE","Nevada":"NV","New Hampshire":"NH",
    "New Jersey":"NJ","New Mexico":"NM","New York":"NY","North Carolina":"NC","North Dakota":"ND","Ohio":"OH",
    "Oklahoma":"OK","Oregon":"OR","Pennsylvania":"PA","Rhode Island":"RI","South Carolina":"SC","South Dakota":"SD",
    "Tennessee":"TN","Texas":"TX","Utah":"UT","Vermont":"VT","Virginia":"VA","Washington":"WA","West Virginia":"WV",
    "Wisconsin":"WI","Wyoming":"WY"
}

if "origin_state" in df.columns:
    df["origin_state"] = df["origin_state"].astype(str).str.strip()
    df["origin_state_abbr"] = df["origin_state"].map(US_STATE_TO_ABBR)

if "destination_state" in df.columns:
    df["destination_state"] = df["destination_state"].astype(str).str.strip()
    df["destination_state_abbr"] = df["destination_state"].map(US_STATE_TO_ABBR)


#### 5.2 Delay KPIs & Categorical Fields
Create:
- `is_operated` (exclude cancelled/diverted flights for KPIs)
- `is_delayed_15` (standard delay definition)
- `arrival_delay_min` (non-negative KPI delay)
- `total_delay_min` (sum of delay causes)
- `delay_bucket` (distribution)
- `primary_delay_cause` (for cause breakdown charts)

In [86]:
cause_cols = [
    "carrier_delay_min",
    "weather_delay_min",
    "nas_delay_min",
    "security_delay_min",
    "late_aircraft_delay_min"
]
cause_cols = [c for c in cause_cols if c in df.columns]  # safety

# Operated flights (exclude cancelled/diverted)
if "is_cancelled" in df.columns and "is_diverted" in df.columns:
    df["is_operated"] = (df["is_cancelled"] == 0) & (df["is_diverted"] == 0)
else:
    df["is_operated"] = True

# KPI-friendly delays (clip negatives)
df["arrival_delay_min"] = df["arrival_delay_raw_min"].clip(lower=0)
df["departure_delay_min"] = df["departure_delay_raw_min"].clip(lower=0)

# Standard delay flag (>15 min)
df["is_delayed_15"] = df["is_operated"] & (df["arrival_delay_raw_min"] > 15)

# Total delay minutes from causes
if cause_cols:
    df["total_delay_min"] = df[cause_cols].sum(axis=1)
else:
    df["total_delay_min"] = 0

# Delay bucket
bins = [-1, 15, 30, 60, 120, 10_000]
labels = ["On time (≤15)", "16–30", "31–60", "61–120", "120+"]
df["delay_bucket"] = pd.cut(df["arrival_delay_raw_min"].clip(lower=-1), bins=bins, labels=labels)

# Primary delay cause
cause_label = {
    "carrier_delay_min": "Carrier",
    "weather_delay_min": "Weather",
    "nas_delay_min": "NAS",
    "security_delay_min": "Security",
    "late_aircraft_delay_min": "Late Aircraft"
}

if cause_cols:
    max_cause = df[cause_cols].idxmax(axis=1)
    df["primary_delay_cause"] = np.where(df["total_delay_min"] > 0, max_cause.map(cause_label), "No Delay")
else:
    df["primary_delay_cause"] = "No Delay"

# Country for consistency
df["country"] = "United States"

df.head()

,flight_date,operating_airline,origin_airport,origin_city,origin_state,destination_airport,destination_city,destination_state,scheduled_departure_hhmm,departure_delay_raw_min,scheduled_arrival_hhmm,arrival_delay_raw_min,is_cancelled,is_diverted,carrier_delay_min,weather_delay_min,nas_delay_min,security_delay_min,late_aircraft_delay_min,year,month,month_name,day_of_month,day_of_week_name,week_of_year,scheduled_departure_hour,scheduled_departure_time,origin_state_abbr,destination_state_abbr,is_operated,arrival_delay_min,departure_delay_min,is_delayed_15,total_delay_min,delay_bucket,primary_delay_cause,country
0,2024-04-18,MQ,DFW,"Dallas/Fort Worth, TX",Texas,RAP,"Rapid City, SD",South Dakota,1018,-3.00,1149,-10.00,0,0,0,0,0,0,0,2024,4,April,18,Thursday,16,10,10:18,TX,SD,True,0.00,0.00,False,0,NaN,No Delay,United States
1,2024-01-01,AA,CLT,"Charlotte, NC",North Carolina,PHX,"Phoenix, AZ",Arizona,1637,-4.00,1923,-17.00,0,0,0,0,0,0,0,2024,1,January,1,Monday,1,16,16:37,NC,AZ,True,0.00,0.00,False,0,NaN,No Delay,United States
2,2024-12-12,9E,CHA,"Chattanooga, TN",Tennessee,ATL,"Atlanta, GA",Georgia,1000,-8.00,1059,-17.00,0,0,0,0,0,0,0,2024,12,December,12,Thursday,50,10,10:00,TN,GA,True,0.00,0.00,False,0,NaN,No Delay,United States
3,2024-04-08,WN,OMA,"Omaha, NE",Nebraska,LAS,"Las Vegas, NV",Nevada,1330,4.00,1430,1.00,0,0,0,0,0,0,0,2024,4,April,8,Monday,15,13,13:30,NE,NV,True,1.00,4.00,False,0,On time (≤15),No Delay,United States
4,2024-02-16,WN,BWI,"Baltimore, MD",Maryland,MYR,"Myrtle Beach, SC",South Carolina,1340,-7.00,1510,-1.00,0,0,0,0,0,0,0,2024,2,February,16,Friday,7,13,13:40,MD,SC,True,0.00,0.00,False,0,NaN,No Delay,United States


---
### 6. Export Cleaned Dataset

We export the cleaned **sample** dataset to `data/processed/flight_clean_data_2024_sample.csv`.

This file is used as a lightweight, reproducible data source for local development and validating the dashboard pipeline.

In [87]:
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

OUT_PATH = PROCESSED_DIR / "flight_clean_data_2024_sample.csv"
df.to_csv(OUT_PATH, index=False)

print("Exported:", OUT_PATH)
print("Final shape:", df.shape)

Exported: ../data/processed/flight_clean_data_2024_sample.csv
Final shape: (10000, 37)
